In [1]:
import myfm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from myfm.utils.benchmark_data import MovieLens100kDataManager
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import scipy.sparse as sps
from itertools import combinations
import numpy as np

In [2]:
data_manager=MovieLens100kDataManager()

In [3]:
# print(dataset.load_rating_predefined_split(3))
ratings = data_manager.load_rating_all().drop(columns=['timestamp'])
users = data_manager.load_user_info()
columns_users = []
for col in users.columns:
    if str(col) != 'user_id' :
        columns_users.append('user_'+str(col))
    else:
        columns_users.append(str(col))
users = pd.DataFrame(users.values,columns=columns_users)
#Merge tập ratings và users được gom nhóm bởi user_id
result =  pd.merge(ratings, users, on="user_id", how="outer")

movies = data_manager.load_movie_info()
columns_movies = []
for col in movies[['movie_id','release_date','genres']].columns:
    if str(col) != 'movie_id' :
        columns_movies.append('movie_'+str(col))
    else:
        columns_movies.append(str(col))
movies = pd.DataFrame(movies[['movie_id','release_date','genres']].values,columns=columns_movies)
result=  pd.merge(result, movies, on="movie_id", how="outer")
result['movie_release_year'] = [str(x) for x in result['movie_release_date'].dt.year.fillna('NaN')]
result["user_age"] = result.user_age // 5 * 5
result["user_zipcode"] = result.user_zipcode.str[0]
df_train,df_test = train_test_split(result, test_size=0.2)


In [81]:
FEATURE_COLUMNS = ['user_id', 'movie_id']

ohe = OneHotEncoder(handle_unknown='ignore')

X_train = ohe.fit_transform(df_train[FEATURE_COLUMNS])
X_test = ohe.transform(df_test[FEATURE_COLUMNS])
y_train = df_train.rating.values
y_test = df_test.rating.values

user_feature_name = [col for col in result.columns if 'user' in col]
user_info = result[user_feature_name].drop_duplicates(subset = "user_id").set_index('user_id')
user_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(user_info)

movie_feature_name = [col for col in result.columns if 'movie' in col]
movie_info = result[movie_feature_name].drop_duplicates(subset = "movie_id").set_index('movie_id')
movie_info = movie_info[['movie_release_year', 'movie_genres']]
movie_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(movie_info[['movie_release_year']])
movie_genre_mle = MultiLabelBinarizer(sparse_output=True).fit(
    movie_info.movie_genres.apply(lambda x: x.split('|'))
)

X_train_extended = sps.hstack([
    X_train,
    user_info_ohe.transform(
        user_info.reindex(df_train.user_id)
    ),
    movie_info_ohe.transform(
        movie_info.reindex(df_train.movie_id).drop(columns=['movie_genres'])
    ),
    movie_genre_mle.transform(
        movie_info.movie_genres.reindex(df_train.movie_id).apply(lambda x: x.split('|'))
    )
])

X_test_extended = sps.hstack([
    X_test,
    user_info_ohe.transform(
        user_info.reindex(df_test.user_id)
    ),
    movie_info_ohe.transform(
        movie_info.reindex(df_test.movie_id).drop(columns=['movie_genres'])
    ),
    movie_genre_mle.transform(
        movie_info.movie_genres.reindex(df_test.movie_id).apply(lambda x: x.split('|'))
    )
])

group_shapes_extended = (
    [len(group) for group in ohe.categories_] +
    [len(group) for group in user_info_ohe.categories_] +
    [len(group) for group in movie_info_ohe.categories_] +
    [len(movie_genre_mle.classes_)]
)
print(group_shapes_extended)
fm_side_info = myfm.MyFMRegressor(
    rank=10, random_seed=42,
)
fm_side_info.fit(
    X_train_extended, y_train, n_iter=150, n_kept_samples=150,
    group_shapes=group_shapes_extended
)

prediction_side_info = fm_side_info.predict(X_test_extended)
rmse = ((y_test - prediction_side_info) ** 2).mean() ** .5
mae = np.abs(y_test - prediction_side_info).mean()
print(f'rmse={rmse}, mae={mae}')

c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


[943, 1651, 14, 2, 21, 19, 72, 19]


alpha = 1.62 w0 = 3.74 : 100%|██████████| 150/150 [00:35<00:00,  4.20it/s]


rmse=0.8878982749575526, mae=0.6980833096814305


In [4]:
# function to generate all the sub lists
def sub_lists (l):
    # initializing empty list
    comb = []
     
    #Iterating till length of list
    for i in range(len(l)+1):
        # Generating sub list
        comb += [list(j) for j in combinations(l, i)]
    # Returning list
    return comb

In [5]:
sub_lists(result.columns.values)

[[],
 ['user_id'],
 ['movie_id'],
 ['rating'],
 ['user_age'],
 ['user_gender'],
 ['user_occupation'],
 ['user_zipcode'],
 ['movie_release_date'],
 ['movie_genres'],
 ['movie_release_year'],
 ['user_id', 'movie_id'],
 ['user_id', 'rating'],
 ['user_id', 'user_age'],
 ['user_id', 'user_gender'],
 ['user_id', 'user_occupation'],
 ['user_id', 'user_zipcode'],
 ['user_id', 'movie_release_date'],
 ['user_id', 'movie_genres'],
 ['user_id', 'movie_release_year'],
 ['movie_id', 'rating'],
 ['movie_id', 'user_age'],
 ['movie_id', 'user_gender'],
 ['movie_id', 'user_occupation'],
 ['movie_id', 'user_zipcode'],
 ['movie_id', 'movie_release_date'],
 ['movie_id', 'movie_genres'],
 ['movie_id', 'movie_release_year'],
 ['rating', 'user_age'],
 ['rating', 'user_gender'],
 ['rating', 'user_occupation'],
 ['rating', 'user_zipcode'],
 ['rating', 'movie_release_date'],
 ['rating', 'movie_genres'],
 ['rating', 'movie_release_year'],
 ['user_age', 'user_gender'],
 ['user_age', 'user_occupation'],
 ['user_age

In [5]:
def best_user_feature(result,df_train,df_test):
    ohe = OneHotEncoder(handle_unknown='ignore')
    FEATURE_COLUMNS = ['user_id', 'movie_id']
    X_train = ohe.fit_transform(df_train[FEATURE_COLUMNS])
    X_test = ohe.transform(df_test[FEATURE_COLUMNS])
    y_train = df_train.rating.values
    y_test = df_test.rating.values
    user_feature_name = [col for col in result.columns if 'user' in col]
    user_data = result[user_feature_name].drop_duplicates(subset = "user_id").set_index('user_id')
    sub_lists_user = sub_lists(user_data.columns)
    minRMSE=float('inf')
    bestfeature = []
    for i in sub_lists_user:
        currentData = user_data.loc[:,i]
        # print(currentData.values.shape)
        currentData_ohe = OneHotEncoder(handle_unknown='ignore').fit(currentData)
        X_train_extended = sps.hstack([
            X_train,
            currentData_ohe.transform(
                currentData.reindex(df_train.user_id)
            ),
        ])

        X_test_extended = sps.hstack([
            X_test,
            currentData_ohe.transform(
                currentData.reindex(df_test.user_id)
            ),
        ])
        group_shapes_extended = (
            [len(group) for group in ohe.categories_] +
            [len(group) for group in currentData_ohe.categories_] 
        )
        print(group_shapes_extended)
        fm_side_info = myfm.MyFMRegressor(
            rank=10, random_seed=42,
        )
        fm_side_info.fit(
            X_train_extended, y_train, n_iter=150, n_kept_samples=150,
            group_shapes=group_shapes_extended
        )

        prediction_side_info = fm_side_info.predict(X_test_extended)
        rmse = ((y_test - prediction_side_info) ** 2).mean() ** .5
        mae = np.abs(y_test - prediction_side_info).mean()
        print(f'rmse={rmse}, mae={mae}')
        if minRMSE > rmse:
            minRMSE=rmse
            bestfeature=i
    print (minRMSE)
    print (bestfeature)
    return bestfeature

In [27]:
def best_movie_feature(result,df_train,df_test):
    ohe = OneHotEncoder(handle_unknown='ignore')
    FEATURE_COLUMNS = ['user_id', 'movie_id']
    X_train = ohe.fit_transform(df_train[FEATURE_COLUMNS])
    X_test = ohe.transform(df_test[FEATURE_COLUMNS])
    y_train = df_train.rating.values
    y_test = df_test.rating.values
    movie_feature_name = [col for col in result.columns if 'movie' in col]
    movie_data = result[movie_feature_name].drop_duplicates(subset = "movie_id").set_index('movie_id')
    movie_data = movie_data[['movie_release_year', 'movie_genres']]
    sub_list_movie = sub_lists(movie_data.columns)
    minRMSE=2
    bestfeature = []
    for i in sub_list_movie:
        if 'movie_genres' not in i:
            print(i)
            currentData = movie_data.loc[:,i]
            currentData_ohe = OneHotEncoder(handle_unknown='ignore').fit(currentData)
            X_train_extended = sps.hstack([
                X_train,
                currentData_ohe.transform(
                    currentData.reindex(df_train.movie_id)
                ),

            ])

            X_test_extended = sps.hstack([
                X_test,
                currentData_ohe.transform(
                    currentData.reindex(df_test.movie_id)
                ),

            ])
            group_shapes_extended = (
                [len(group) for group in ohe.categories_] +
                [len(group) for group in currentData_ohe.categories_] 
            )
        else:
            currentData = movie_data.loc[:,i].drop(columns=['movie_genres'])
            movie_genre = movie_data.loc[:,['movie_genres']]
            currentData_ohe = OneHotEncoder(handle_unknown='ignore').fit(currentData)
            print(i)
            movie_genre_mle = MultiLabelBinarizer(sparse_output=True).fit(
            movie_genre.movie_genres.apply(lambda x: x.split('|'))
            )
            # print(movie_genre_mle.classes_)
            X_train_extended = sps.hstack([
                X_train,
                currentData_ohe.transform(
                    currentData.reindex(df_train.movie_id)
                ),
                movie_genre_mle.transform(
                    movie_genre.movie_genres.reindex(df_train.movie_id).apply(lambda x: x.split('|'))
                )
            ])

            X_test_extended = sps.hstack([
                X_test,
                currentData_ohe.transform(
                    currentData.reindex(df_test.movie_id)
                ),
                movie_genre_mle.transform(
                    movie_genre.movie_genres.reindex(df_test.movie_id).apply(lambda x: x.split('|'))
                )
            ])
            group_shapes_extended = (
                [len(group) for group in ohe.categories_] +
                [len(group) for group in currentData_ohe.categories_] +
                [len(movie_genre_mle.classes_)]
            )
        print(group_shapes_extended)
        fm_side_info = myfm.MyFMRegressor(
            rank=10, random_seed=42,
        )
        fm_side_info.fit(
            X_train_extended, y_train, n_iter=150, n_kept_samples=150,
            group_shapes=group_shapes_extended
        )

        prediction_side_info = fm_side_info.predict(X_test_extended)
        rmse = ((y_test - prediction_side_info) ** 2).mean() ** .5
        mae = np.abs(y_test - prediction_side_info).mean()
        print(f'rmse={rmse}, mae={mae}')
        if minRMSE > rmse:
            minRMSE=rmse
            bestfeature=i
    print (minRMSE)
    print (bestfeature)
    return bestfeature

In [84]:
def best_features(result,df_train,df_test,full_feature_rmse):
    ohe = OneHotEncoder(handle_unknown='ignore')
    FEATURE_COLUMNS = ['user_id', 'movie_id']
    X_train = ohe.fit_transform(df_train[FEATURE_COLUMNS])
    X_test = ohe.transform(df_test[FEATURE_COLUMNS])
    y_train = df_train.rating.values
    y_test = df_test.rating.values

    user_feature_name = [col for col in result.columns if 'user' in col]
    user_data = result[user_feature_name].drop_duplicates(subset = "user_id").set_index('user_id')
    sub_lists_user = sub_lists(user_data.columns)

    movie_feature_name = [col for col in result.columns if 'movie' in col]
    movie_data = result[movie_feature_name].drop_duplicates(subset = "movie_id").set_index('movie_id')
    movie_data = movie_data[['movie_release_year', 'movie_genres']]
    sub_list_movie = sub_lists(movie_data.columns)

    minRMSE=full_feature_rmse
    bestfeature_movie = []
    bestfeature_user = []

    for i in sub_list_movie:
        if len(i)>0:
            if 'movie_genres' not in i:
                print(i)
                current_movie_Data = movie_data.loc[:,i]
                current_movie_Data_ohe = OneHotEncoder(handle_unknown='ignore').fit(current_movie_Data)
                for j in sub_lists_user:
                    if len(j)>0:
                        current_user_Data = user_data.loc[:,j]
                        current_user_Data_ohe = OneHotEncoder(handle_unknown='ignore').fit(current_user_Data)
                        X_train_extended = sps.hstack([
                            X_train,
                            current_user_Data_ohe.transform(
                                current_user_Data.reindex(df_train.user_id)
                            ),
                            current_movie_Data_ohe.transform(
                            current_movie_Data.reindex(df_train.movie_id)
                        ),
                        ])

                        X_test_extended = sps.hstack([
                            X_test,
                            current_user_Data_ohe.transform(
                            current_user_Data.reindex(df_test.user_id)
                            ),
                            current_movie_Data_ohe.transform(
                            current_movie_Data.reindex(df_test.movie_id)
                        ),
                        ])
                        group_shapes_extended = (
                            [len(group) for group in ohe.categories_] +
                            [len(group) for group in current_user_Data_ohe.categories_] +
                            [len(group) for group in current_movie_Data_ohe.categories_] 
                        )
                        print(group_shapes_extended)
                        fm_side_info = myfm.MyFMRegressor(
                            rank=10, random_seed=42,
                        )
                        fm_side_info.fit(
                            X_train_extended, y_train, n_iter=150, n_kept_samples=150,
                            group_shapes=group_shapes_extended
                        )

                        prediction_side_info = fm_side_info.predict(X_test_extended)
                        rmse = ((y_test - prediction_side_info) ** 2).mean() ** .5
                        mae = np.abs(y_test - prediction_side_info).mean()
                        print(f'rmse={rmse}, mae={mae}')
                        if minRMSE > rmse:
                            minRMSE=rmse
                            bestfeature_user=j
                            bestfeature_movie =i
                            break
            else:
                current_movie_Data = movie_data.loc[:,i].drop(columns=['movie_genres'])
                movie_genre = movie_data.loc[:,['movie_genres']]
                current_movie_Data_ohe = OneHotEncoder(handle_unknown='ignore').fit(current_movie_Data)
                print(i)
                movie_genre_mle = MultiLabelBinarizer(sparse_output=True).fit(
                movie_genre.movie_genres.apply(lambda x: x.split('|'))
                )
                for j in sub_lists_user:
                    if len(j) > 0:
                        current_user_Data = user_data.loc[:,j]
                        current_user_Data_ohe = OneHotEncoder(handle_unknown='ignore').fit(current_user_Data)
                        X_train_extended = sps.hstack([
                            X_train,
                            current_user_Data_ohe.transform(
                                current_user_Data.reindex(df_train.user_id)
                            ),
                            current_movie_Data_ohe.transform(
                            current_movie_Data.reindex(df_train.movie_id)
                        ),
                            movie_genre_mle.transform(
                            movie_genre.movie_genres.reindex(df_train.movie_id).apply(lambda x: x.split('|'))
                        )
                        ])

                        X_test_extended = sps.hstack([
                            X_test,
                            current_user_Data_ohe.transform(
                            current_user_Data.reindex(df_test.user_id)
                            ),
                            current_movie_Data_ohe.transform(
                            current_movie_Data.reindex(df_test.movie_id)
                        ),
                            movie_genre_mle.transform(
                            movie_genre.movie_genres.reindex(df_test.movie_id).apply(lambda x: x.split('|'))
                        )
                        ])
                        group_shapes_extended = (
                            [len(group) for group in ohe.categories_] +
                            [len(group) for group in current_user_Data_ohe.categories_] +
                            [len(group) for group in current_movie_Data_ohe.categories_] +
                            [len(movie_genre_mle.classes_)]
                        )
                        print(group_shapes_extended)
                        fm_side_info = myfm.MyFMRegressor(
                            rank=10, random_seed=42,
                        )
                        fm_side_info.fit(
                            X_train_extended, y_train, n_iter=150, n_kept_samples=150,
                            group_shapes=group_shapes_extended
                        )

                        prediction_side_info = fm_side_info.predict(X_test_extended)
                        rmse = ((y_test - prediction_side_info) ** 2).mean() ** .5
                        mae = np.abs(y_test - prediction_side_info).mean()
                        print(f'rmse={rmse}, mae={mae}')
                        if minRMSE > rmse:
                            minRMSE=rmse
                            bestfeature_user=j
                            bestfeature_movie =i
                            break
    print (minRMSE)
    print (bestfeature_user)
    print (bestfeature_movie)
    return [minRMSE,bestfeature_user,bestfeature_movie]

In [66]:
a = best_user_feature(result,df_train,df_test)
b = best_movie_feature(result,df_train,df_test)


c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


[943, 1658]


alpha = 1.64 w0 = 3.51 : 100%|██████████| 150/150 [00:07<00:00, 19.05it/s]


rmse=0.8953497974408914, mae=0.7052274401030256
[943, 1658, 14]


alpha = 1.63 w0 = 3.56 : 100%|██████████| 150/150 [00:10<00:00, 13.91it/s]


...

rmse=0.894148700450063, mae=0.7039234697317936
['movie_genres']
['Action' 'Adventure' 'Animation' "Children's" 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'Musical' 'Mystery'
 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western' 'unknown']
[943, 1658, 19]


alpha = 1.60 w0 = 3.42 : 100%|██████████| 150/150 [00:14<00:00, 10.34it/s]


rmse=0.8893470535266542, mae=0.6998241077817234
['movie_release_year', 'movie_genres']
['Action' 'Adventure' 'Animation' "Children's" 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'Musical' 'Mystery'
 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western' 'unknown']
[943, 1658, 72, 19]


alpha = 1.62 w0 = 3.49 : 100%|██████████| 150/150 [00:16<00:00,  9.02it/s]


rmse=0.8880186250992825, mae=0.6981632797888159
0.8880186250992825
['movie_release_year', 'movie_genres']


In [51]:
a

['user_gender', 'user_zipcode']

In [52]:
b

['movie_release_year', 'movie_genres']

In [67]:
c = a + b + ['user_id','movie_id','rating']
c

['user_age',
 'user_gender',
 'user_occupation',
 'movie_release_year',
 'movie_genres',
 'user_id',
 'movie_id',
 'rating']

In [68]:

newData = result.loc[:,c]
train = df_train.loc[:,c]
test = df_test.loc[:,c]


In [69]:
aohe= OneHotEncoder(handle_unknown='ignore')


X_train = aohe.fit_transform(train[FEATURE_COLUMNS])
X_test = aohe.transform(test[FEATURE_COLUMNS])
y_train = train.rating.values
y_test = test.rating.values

user_feature_name = [col for col in newData.columns if 'user' in col]
user_info = newData[user_feature_name].drop_duplicates(subset = "user_id").set_index('user_id')
user_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(user_info)

movie_feature_name = [col for col in newData.columns if 'movie' in col]
movie_info = newData[movie_feature_name].drop_duplicates(subset = "movie_id").set_index('movie_id')
# movie_info['release_year'] = [
#     str(x) for x in movie_info['release_date'].dt.year.fillna('NaN')
# ]
movie_info = movie_info[['movie_release_year', 'movie_genres']]
movie_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(movie_info[['movie_release_year']])
movie_genre_mle = MultiLabelBinarizer(sparse_output=True).fit(
    movie_info.movie_genres.apply(lambda x: x.split('|'))
)
movie_genre_mle.classes_
X_train_extended = sps.hstack([
    X_train,
    user_info_ohe.transform(
        user_info.reindex(df_train.user_id)
    ),
    movie_info_ohe.transform(
        movie_info.reindex(df_train.movie_id).drop(columns=['movie_genres'])
    ),
    movie_genre_mle.transform(
        movie_info.movie_genres.reindex(df_train.movie_id).apply(lambda x: x.split('|'))
    )
])

X_test_extended = sps.hstack([
    X_test,
    user_info_ohe.transform(
        user_info.reindex(df_test.user_id)
    ),
    movie_info_ohe.transform(
        movie_info.reindex(df_test.movie_id).drop(columns=['movie_genres'])
    ),
    movie_genre_mle.transform(
        movie_info.movie_genres.reindex(df_test.movie_id).apply(lambda x: x.split('|'))
    )
])

group_shapes_extended = (
    [len(group) for group in aohe.categories_] +
    [len(group) for group in user_info_ohe.categories_] +
    [len(group) for group in movie_info_ohe.categories_] +
    [ len(movie_genre_mle.classes_)]
)

print(group_shapes_extended)

fm_side_info = myfm.MyFMRegressor(
    rank=10, random_seed=42,
)
fm_side_info.fit(
    X_train_extended, y_train, n_iter=150, n_kept_samples=150,
    group_shapes=group_shapes_extended
)

prediction_side_info = fm_side_info.predict(X_test_extended)
rmse = ((y_test - prediction_side_info) ** 2).mean() ** .5
mae = np.abs(y_test - prediction_side_info).mean()
print(f'rmse={rmse}, mae={mae}')

c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


[943, 1658, 14, 2, 21, 72, 19]


alpha = 1.60 w0 = 3.69 : 100%|██████████| 150/150 [00:25<00:00,  5.90it/s]


rmse=0.8871748682178033, mae=0.6970300622335975
